In [ ]:
import seaborn as sns
import polars as pl
import plotly.express as px

import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

# Load up metadata from "DNA from hoiho genomesv2.csv"
metadata = pl.read_csv(
    "Hoiho_Genomes_Cleaned_Nov4_24_24.csv", separator="\t"
)

metadata = metadata.with_columns(
    pl.col("ID").replace("P29 ", "P29").alias("ID")
)

# Next is C101/CE9
metadata = metadata.with_columns(
    pl.col("ID").replace("C101/CE9", "CE9").alias("ID")
)

# bcftools view '-eF_MISSING>0.2 | QUAL < 99' -s^PP6,PP19 -v snps --threads 32 --min-af 0.05:minor -m2 -M2  -Ob -o merged.a9.filtered.qual99_fmissing0.2.maf0.05.biallelic.bcf merged.a9.unfiltered.bcf
# pixi run plink2 --allow-extra-chr --bcf merged.a9.filtered.qual99_fmissing0.2.maf0.05.biallelic.bcf --pca --out merged.a9.filtered.qual99_fmissing0.2.maf0.05.biallelic --vcf-half-call missing

# bcftools view '-eQUAL < 99' -s^PP6,PP19 -v snps --threads 32 --min-af 0.05:minor -m2 -M2  -Ob -o merged.a9.filtered.qual99.maf0.05.biallelic.bcf merged.a9.unfiltered.bcf
# pixi run plink2 --allow-extra-chr --bcf merged.a9.filtered.qual99.maf0.05.biallelic.bcf --pca --out merged.a9.filtered.qual99.maf0.05.biallelic --vcf-half-call missing

eigenvec = pl.read_csv("merged.a9.filtered.qual99_fmissing0.2.maf0.05.biallelic.eigenvec",
  separator="\t")
# eigenvec = pl.read_csv("merged.a9.filtered.qual99.maf0.05.biallelic", separator="\t")
eigenvec = pl.read_csv("removed_qual99_maf0.05.eigenvec", separator="\t")
eigenvec = eigenvec.rename(mapping={"#IID": "ID"})
joined = eigenvec.join(metadata, on="ID", how="inner")

joined = joined.to_pandas()

eigenvec

In [ ]:
# PC2 is split by sex, males are positive, females are negative 
# Create a field called "Sex_Genome"

# Create a field
joined['Sex_Genome'] = joined['PC2'] > 0

# Convert to Male/Female
joined['Sex_Genome'] = joined['Sex_Genome'].map({True: 'Male', False: 'Female'})

In [ ]:
# Save dataframe with ID and Sex_Genome
joined[['ID', 'Sex_Genome', 'Sex']].to_csv("GeneticSex.csv", index=False)

In [ ]:
joined.columns